In [1]:
import pandas as pd

from IPython.display import Markdown

In [23]:
hours_correction = 1.05
hours = 350

In [3]:
years = range(2020,2025)
rates = {
    2020 : {
        100: 0,
        150: 15,
        250: 25,
        350: 30,
        450: 45,
        1000: 55  
    },
    2021 : {
        100: 0,
        150: 29,
        250: 43,
        350: 49,
        450: 61,
        1000: 70        
    },
    2022 : {
        100: 0,
        150: 43,
        250: 60,
        350: 68,
        450: 78,
        1000: 85 
    },
    2023 : {
        100: 0,
        150: 56,
        250: 78,
        350: 86,
        450: 94,
        1000: 100 
    },
    2024 : {
        100: 0,
        150: 70,
        250: 95,
        350: 105,
        450: 110,
        1000: 115  
    }
}

In [4]:
profit_thresholds_2018_CBA = {
    24: 0.5,
    49: 1.2,
    99: 2.0,
}
profit_thresholds_2020_CBA = {
    24: 0.5,
    49: 1.5,
    99: 2.5,
}

profit = {
    2020: 0,
    2021: 10,
    2022: 20,
    2023: 30,
    2024: 40
}
profit_table = pd.DataFrame(profit.values(), columns=['profit_in_M'], index=profit.keys())

In [5]:
def value_of_profit_to_crewmember(profit_in_millions, cba=2020):
    applicable_rate = 0
    
    if cba == 2018:
        profit_thresholds = profit_thresholds_2018_CBA
        profit = profit_in_millions if (profit_in_millions < 100) else 100
    elif cba == 2020:
        profit_thresholds = profit_thresholds_2020_CBA
        profit = profit_in_millions if (profit_in_millions < 150) else 150
        
    
    for threshold, rate in profit_thresholds.items():
        if profit > threshold:
            applicable_rate = rate
        else:
            break
    
    profit = profit * 10**6
    share_for_all_crewmembers = profit * (applicable_rate/100)
    share_for_individual_crewmember = share_for_all_crewmembers / 770
    return share_for_individual_crewmember

value_of_profit_to_crewmember(profit_in_millions=55, cba=2020)

1071.4285714285713

In [6]:
def value_of_hours(hours, year):
    total = 0
    previous_threshold = 0

    for threshold, rate in rates[year].items():
        hours_over_threshold = hours - threshold
        hours_over_previous_threshold = hours - previous_threshold

        if hours_over_threshold > 0:
            total += (threshold - previous_threshold) * rate
        elif hours_over_previous_threshold > 0:
            total += (hours - previous_threshold) * rate

        previous_threshold = threshold

    return total

In [7]:
cps = {
    2020: 6000,
    2021: 6000,
    2022: 9000,
    2023: 9000,
    2024: 9000,
}
senior_cps = {
    2020: 15000,
    2021: 15000,
    2022: 15000,
    2023: 15000,
    2024: 15000
}
cps_factor ={
    2020: 1.0,
    2021: 0.75,
    2022: 0.5,
    2023: 0.25,
    2024: 0,
}

In [8]:
value_of_hours(350, 2024)

23500

In [24]:
cba_2020 = pd.DataFrame()
cba_2020['year'] = years
cba_2020['productivity'] = cba_2020.year.apply(lambda year : value_of_hours(hours, year) )
cba_2020['cps'] = senior_cps.values()
cba_2020['factor'] = cps_factor.values()
cba_2020['new_cps'] = cba_2020.cps * cba_2020.factor
cba_2020['profit_reward'] = cba_2020.year.apply(
    lambda year : value_of_profit_to_crewmember( profit_in_millions=profit[year], cba=2020 )
)
cba_2020.drop(columns=['cps', 'factor'], inplace=True)
cba_2020.set_index('year', inplace=True)
cba_2020 = cba_2020.round(2)

In [10]:
cba_2020['total'] = cba_2020.sum(axis=1)
cba_2020

,productivity,new_cps,profit_reward,total
year,,,,
2020,6250,15000.0,0.00,21250.00
2021,10650,11250.0,0.00,21900.00
2022,14950,7500.0,0.00,22450.00
2023,19200,3750.0,194.81,23144.81
2024,23500,0.0,259.74,23759.74


In [11]:
cba_2020_t = cba_2020.transpose()

cba_2020_t

year,2020,2021,2022,2023,2024
productivity,6250.0,10650.0,14950.0,19200.00,23500.00
new_cps,15000.0,11250.0,7500.0,3750.00,0.00
profit_reward,0.0,0.0,0.0,194.81,259.74
total,21250.0,21900.0,22450.0,23144.81,23759.74


In [25]:
cba_2018 = pd.DataFrame()
cba_2018['year'] = years
cba_2018['productivity'] = cba_2018.year.apply(lambda year : value_of_hours(hours, 2020) )
cba_2018['cps'] = senior_cps.values()
cba_2018['profit_reward'] = cba_2018.year.apply(
    lambda year : value_of_profit_to_crewmember( profit_in_millions=profit[year], cba=2018 )
)
cba_2018.set_index('year', inplace=True)
cba_2018['total'] = cba_2018.sum(axis=1)
cba_2018 = cba_2018.round(2)
cba_2018_t = cba_2018.transpose()

year,2020,2021,2022,2023,2024
productivity,6250.0,6250.0,6250.0,6250.00,6250.00
cps,15000.0,15000.0,15000.0,15000.00,15000.00
profit_reward,0.0,0.0,0.0,194.81,259.74
total,21250.0,21250.0,21250.0,21444.81,21509.74


In [13]:
profit_table.transpose()

,2020,2021,2022,2023,2024
profit_in_M,0,10,20,30,40


In [14]:
cba_2018_t

year,2020,2021,2022,2023,2024
productivity,6250.0,6250.0,6250.0,6250.00,6250.00
cps,15000.0,15000.0,15000.0,15000.00,15000.00
profit_reward,0.0,0.0,0.0,194.81,259.74
total,21250.0,21250.0,21250.0,21444.81,21509.74


In [26]:
display(Markdown("## Hours"))
display(hours)
display(Markdown("## 2020 CBA"))
display(cba_2020_t)
display(Markdown("## 2018 CBA"))
display(cba_2018_t)

## Hours

350

## 2020 CBA

year,2020,2021,2022,2023,2024
productivity,6250.0,10650.0,14950.0,19200.00,23500.00
new_cps,15000.0,11250.0,7500.0,3750.00,0.00
profit_reward,0.0,0.0,0.0,194.81,259.74
total,21250.0,21900.0,22450.0,23144.81,23759.74


## 2018 CBA

year,2020,2021,2022,2023,2024
productivity,6250.0,6250.0,6250.0,6250.00,6250.00
cps,15000.0,15000.0,15000.0,15000.00,15000.00
profit_reward,0.0,0.0,0.0,194.81,259.74
total,21250.0,21250.0,21250.0,21444.81,21509.74


## Tour Based Pay

In [35]:
class FW:
    def __init__(self, name, salary_factor, feb_may_days, jun_sep_days, oct_jan_days, vacation=26):
        self.name = name
        self.salary_factor = salary_factor
        self.feb_may_days = feb_may_days
        self.jun_sep_days = jun_sep_days
        self.oct_jan_days = oct_jan_days
        self.vacation = vacation
        self.base_salary = 83000
        
    def days_per_month(self):
        vacation_per_month = self.vacation / 12
        days_per_month = [
            self.oct_jan_days / 4,
            self.feb_may_days / 4,
            self.feb_may_days / 4,
            self.feb_may_days / 4,
            self.feb_may_days / 4,
            self.jun_sep_days / 4,
            self.jun_sep_days / 4,
            self.jun_sep_days / 4,
            self.jun_sep_days / 4,
            self.oct_jan_days / 4,
            self.oct_jan_days / 4,
            self.oct_jan_days / 4,           
        ]
        days_per_month = [round(days - vacation_per_month, 2) for days in days_per_month]
        return days_per_month
    
    def salary(self):
        return self.base_salary * self.salary_factor
    
    def TBP(self):
        if self.name == 'FW200':
            return 0
        return (self.jun_sep_days - (self.vacation / 3)) * (2.3 - 1.52) * 105
    

month_names = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

FW200 = FW('FW200', 1.0, 66.68, 66.68, 66.68)
FW204 = FW('FW204', 1.025, 66, 72, 66)
FW208 = FW('FW208', 1.05, 66, 76, 66)
FW212 = FW('FW212', 1.075, 68, 76, 68)
FW216 = FW('FW216', 1.1, 70, 76, 70)
FW220 = FW('FW220', 1.125, 72, 76, 72)

rosters = [FW200, FW204, FW208, FW212, FW216, FW220]

In [36]:
flex_work = pd.DataFrame()
flex_work['month'] = month_names
flex_work.set_index('month', inplace=True)

for roster in rosters:
    flex_work[roster.name] = roster.days_per_month()
    

hours = flex_work * 2.3
hours_t = hours.transpose()
hours_t['total_hours'] = hours_t.sum(axis=1)
hours_t['productivity'] = hours_t.total_hours.apply(lambda hours: value_of_hours(hours, 2024))

salary = [roster.salary() for roster in rosters]

hours_t['tbp'] = [roster.TBP() for roster in rosters]

hours_t['salary'] = salary

hours_t['total_recomp'] = hours_t.productivity + hours_t.tbp + hours_t.salary

hours_t

month,January,February,March,April,May,June,July,August,September,October,November,December,total_hours,productivity,tbp,salary,total_recomp
FW200,33.350,33.350,33.350,33.350,33.350,33.350,33.350,33.350,33.350,33.350,33.350,33.350,400.200,29022.00,0.0,83000.0,112022.00
FW204,32.959,32.959,32.959,32.959,32.959,36.409,36.409,36.409,36.409,32.959,32.959,32.959,409.308,30023.88,5187.0,85075.0,120285.88
FW208,32.959,32.959,32.959,32.959,32.959,38.709,38.709,38.709,38.709,32.959,32.959,32.959,418.508,31035.88,5514.6,87150.0,123700.48
FW212,34.109,34.109,34.109,34.109,34.109,38.709,38.709,38.709,38.709,34.109,34.109,34.109,427.708,32047.88,5514.6,89225.0,126787.48
FW216,35.259,35.259,35.259,35.259,35.259,38.709,38.709,38.709,38.709,35.259,35.259,35.259,436.908,33059.88,5514.6,91300.0,129874.48
FW220,36.409,36.409,36.409,36.409,36.409,38.709,38.709,38.709,38.709,36.409,36.409,36.409,446.108,34071.88,5514.6,93375.0,132961.48


In [37]:
hours = hours_t.transpose()
hours

,FW200,FW204,FW208,FW212,FW216,FW220
month,,,,,,
January,33.35,32.959,32.959,34.109,35.259,36.409
February,33.35,32.959,32.959,34.109,35.259,36.409
March,33.35,32.959,32.959,34.109,35.259,36.409
April,33.35,32.959,32.959,34.109,35.259,36.409
May,33.35,32.959,32.959,34.109,35.259,36.409
June,33.35,36.409,38.709,38.709,38.709,38.709
July,33.35,36.409,38.709,38.709,38.709,38.709
August,33.35,36.409,38.709,38.709,38.709,38.709
September,33.35,36.409,38.709,38.709,38.709,38.709


In [19]:
class SummerTour:
    
    rates_per_tour = {
        2: {
            'Days': 2,
            'Threshold' : 1.1,
            'Rate per hour': 105
        },
        3: {
            'Days': 3,
            'Threshold' : 3.1,
            'Rate per hour': 105
        },
        4: {
            'Days': 4,
            'Threshold' : 5.1,
            'Rate per hour': 105
        },
        5: {
            'Days': 5,
            'Threshold' : 7.1,
            'Rate per hour': 105
        },
        6: {
            'Days': 6,
            'Threshold' : 9.1,
            'Rate per hour': 105
        },
        7: {
            'Days': 7,
            'Threshold' : 11.1,
            'Rate per hour': 105
        },
    }
    
    def __init__(self, days, hours):
        self.days = days
        self.hours = hours * hours_correction
        
    def value(self):
        threshold = self.rates_per_tour[self.days]['Threshold']
        rate = self.rates_per_tour[self.days]['Rate per hour']
        hours_over_threshold = self.hours - threshold
        hours_over_threshold = hours_over_threshold if hours_over_threshold > 0 else 0
        value = hours_over_threshold * rate
        return round(value, 2)

## Tour Based Pay

In [20]:
TBP_2021 = []
TBP_2021.append(SummerTour(days=6, hours=12))

total=0
for tour in TBP_2021:
    total+=tour.value()
    
total

367.5

In [29]:
64/6 * 367.5

3920.0